In [40]:
!pip install nltk


   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ------------- -------------------------- 0.5/1.5 MB 1.9 MB/s eta 0:00:01
   ------------- -------------------------- 0.5/1.5 MB 1.9 MB/s eta 0:00:01
   -------------------- ------------------- 0.8/1.5 MB 1.3 MB/s eta 0:00:01
   --------------------------- ------------ 1.0/1.5 MB 1.3 MB/s eta 0:00:01
   ---------------------------------- ----- 1.3/1.5 MB 1.3 MB/s eta 0:00:01
   ---------------------------------------- 1.5/1.5 MB 1.3 MB/s eta 0:00:00


In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.preprocessing import FunctionTransformer

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd
import numpy as np
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import nltk # Download the punkt tokenizer 
nltk.download('punkt') 
# nltk.df.path.append(r'D:\Work file\bd_real_estate\real_estate_venv')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [203]:
df = pd.read_csv(r"D:\Work file\bd_real_estate\notebooks\Recommendation_data.csv", index_col=0)

In [204]:
df.head()

,property_name,address,short_description,price,bedrooms,bathrooms,floor_area,property_url,type,teg,price_per_sqft,sub_area,area
0,Flat For Sale In Agargaon Close To Agargaon Ja...,"West Agargaon, Agargaon",lovely apartment is vacant at the moment you g...,8500000.0,3.0,3.0,1275.0,https://www.bproperty.com/buy/dhaka/agargaon/f...,buy,agargaon,6666.666667,West Agargaon,Agargaon
1,We Have A 1133 Sq.Ft Flat For Sale In The Loca...,"West Agargaon, Agargaon",the vacant flat surely is a reasonable deal co...,6231500.0,3.0,3.0,1133.0,https://www.bproperty.com/buy/dhaka/agargaon/w...,buy,agargaon,5500.000000,West Agargaon,Agargaon
2,A Must See This Apartment For Sale Is All Set ...,"Science Museum To Water Tank Road, Agargaon",looking for a suitable place to settle in the ...,7800000.0,3.0,3.0,1356.0,https://www.bproperty.com/buy/dhaka/agargaon/a...,buy,agargaon,5752.212389,Science Museum To Water Tank Road,Agargaon
3,Affordableble 765 Sq Ft Apartment Is Up For Sa...,"West Agargaon, Agargaon",a wellconstructed and nicely planned flat is v...,5207500.0,2.0,2.0,765.0,https://www.bproperty.com/buy/dhaka/agargaon/a...,buy,agargaon,6807.189542,West Agargaon,Agargaon
4,Wonderful 895 SQ FT home is available for sale...,"West Agargaon, Agargaon",good news for the home hunters if you are plan...,5500000.0,2.0,2.0,895.0,https://www.bproperty.com/buy/dhaka/agargaon/w...,buy,agargaon,6145.251397,West Agargaon,Agargaon


In [205]:
df.dropna(inplace=True)

In [206]:
df['area'].value_counts()

area
Mirpur             1565
Bashundhara R-A     876
Uttara              605
Badda               448
Mohammadpur         341
Banasree            284
Aftab Nagar         216
Dakshin Khan        205
Dhanmondi           149
Agargaon            111
Rampura             103
Khilgaon             87
Tejgaon              57
Motijheel            54
Gulshan              48
Mohakhali            32
Keraniganj           30
New Market           27
Banani               24
Name: count, dtype: int64

In [207]:
df.isnull().sum()

property_name        0
address              0
short_description    0
price                0
bedrooms             0
bathrooms            0
floor_area           0
property_url         0
type                 0
teg                  0
price_per_sqft       0
sub_area             0
area                 0
dtype: int64

In [208]:
df.shape

(5262, 13)

In [209]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5262 entries, 0 to 5298
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   property_name      5262 non-null   object 
 1   address            5262 non-null   object 
 2   short_description  5262 non-null   object 
 3   price              5262 non-null   float64
 4   bedrooms           5262 non-null   float64
 5   bathrooms          5262 non-null   float64
 6   floor_area         5262 non-null   float64
 7   property_url       5262 non-null   object 
 8   type               5262 non-null   object 
 9   teg                5262 non-null   object 
 10  price_per_sqft     5262 non-null   float64
 11  sub_area           5262 non-null   object 
 12  area               5262 non-null   object 
dtypes: float64(5), object(8)
memory usage: 575.5+ KB


In [218]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, FunctionTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd
import numpy as np
import spacy
nlp = spacy.load('en_core_web_sm')

# Custom transformer for text preprocessing with lemmatization
class TextPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self, nlp):
        self.nlp = nlp

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X.apply(lambda text: ' '.join([token.lemma_ for token in self.nlp(str(text))]))


In [220]:
# Text and numerical processing pipeline
text_pipeline = Pipeline([
    ('text_preprocessing', TextPreprocessor(nlp)),
    ('tfidf', TfidfVectorizer(stop_words='english', max_df=0.7, min_df=0.01))
])


In [221]:
# Categorical encoding pipeline
area_pipeline = Pipeline([
    ('ohe', OneHotEncoder(handle_unknown='ignore'))
])


In [222]:
# Numerical processing with additional interaction features
def add_interaction_features(df):
    df['price_per_sqft'] = df['price'] / df['floor_area']
    df['bed_bath_ratio'] = df['bedrooms'] / (df['bathrooms'] + 1)  # Adding 1 to avoid division by zero
    return df[['price', 'bedrooms', 'bathrooms', 'floor_area', 'price_per_sqft', 'bed_bath_ratio']]

numeric_pipeline = Pipeline([
    ('interaction_features', FunctionTransformer(add_interaction_features, validate=False)),
    ('scaler', MinMaxScaler())
])

In [223]:
# Combine all features
preprocessor = ColumnTransformer([
    ('text', text_pipeline, 'short_description'),
    ('area', area_pipeline, ['area']),
    ('numeric', numeric_pipeline, ['price', 'bedrooms', 'bathrooms', 'floor_area'])
])

In [224]:
# Fit and transform data
combined_features = preprocessor.fit_transform(df)

In [225]:
similarity_matrix = cosine_similarity(combined_features)

In [64]:
# # Weighted cosine similarity function
# def weighted_cosine_similarity(matrix, weights):
#     # Apply weights along each feature
#     weighted_matrix = matrix * weights.reshape(1, -1)
#     return cosine_similarity(weighted_matrix)

In [65]:
# similarity_matrix = weighted_cosine_similarity(combined_features_dense, weights)


In [231]:
# Filter function to narrow down the dataset
def filter_data(data, area=None, min_price=None, max_price=None, min_bedrooms=None, max_bedrooms=None):
    filtered_data = data.copy()
    if area:
        filtered_data = filtered_data[filtered_data['area'].str.contains(area, case=False)]
    if min_price is not None:
        filtered_data = filtered_data[filtered_data['price'] >= min_price]
    if max_price is not None:
        filtered_data = filtered_data[filtered_data['price'] <= max_price]
    if min_bedrooms is not None:
        filtered_data = filtered_data[filtered_data['bedrooms'] >= min_bedrooms]
    if max_bedrooms is not None:
        filtered_data = filtered_data[filtered_data['bedrooms'] <= max_bedrooms]
    return filtered_data

# Function to get recommendations based on similarity, now with area and other filters
def get_recommendations(index, top_n=5, area=None, min_price=None, max_price=None, min_bedrooms=None, max_bedrooms=None):
    filtered_data = filter_data(df, area, min_price, max_price, min_bedrooms, max_bedrooms)
    filtered_indices = filtered_data.index.tolist()
    
    # Calculate similarity only for filtered data
    similarity_scores = [(i, similarity_matrix[index][i]) for i in filtered_indices]
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    top_properties = [i[0] for i in similarity_scores[1:top_n+1]]
    
    return df[['property_name', 'price', 'bedrooms', 'bathrooms', 'floor_area', 'area', 'short_description', 'property_url']].iloc[top_properties]

# Display recommendations function with additional details
def display_recommendations(index, top_n=5, area=None, min_price=None, max_price=None, min_bedrooms=None, max_bedrooms=None):
    recommendations = get_recommendations(index, top_n, area, min_price, max_price, min_bedrooms, max_bedrooms)
    print("Top Recommendations:\n")
    for i, row in recommendations.iterrows():
        print(f"Property Name: {row['property_name']}")
        print(f"Location (Area): {row['area']}")
        print(f"Price: {row['price']}")
        print(f"Floor Area: {row['floor_area']} sqft")
        print(f"Bedrooms: {row['bedrooms']}, Bathrooms: {row['bathrooms']}")
        print(f"Description: {row['short_description'][:100]}...")  # Display first 100 characters
        print(f"URL: {row['property_url']}")
        print("-" * 40)

# Example usage with filters
display_recommendations(0, top_n=5, area='Mirpur', min_price=9000000, max_price=10000000, min_bedrooms=2, max_bedrooms=3)

Top Recommendations:

Property Name: Visit This Apartment For Sale In Mirpur Near Mirpur Adhunik Hospital And Diagnostic C...
Location (Area):  Mirpur
Price: 9000000.0
Floor Area: 1250.0 sqft
Bedrooms: 3.0, Bathrooms: 3.0
Description: lovely apartment is vacant at the moment you get all the facilities you need along with a spacious p...
URL: https://www.bproperty.com/buy/dhaka/mirpur/visit-this-apartment-for-sale-in-mirpur-near-mirpu-171763838323/
----------------------------------------
Property Name: Buy This 1340 Sq Ft Apartment In Eastern Pallabi, Mirpur
Location (Area):  Mirpur
Price: 9000000.0
Floor Area: 1340.0 sqft
Bedrooms: 3.0, Bathrooms: 3.0
Description: you have landed yourself in the right listing if you are looking for one of the most picked and affo...
URL: https://www.bproperty.com/buy/dhaka/mirpur/buy-this-1340-sq-ft-apartment-in-eastern-pallabi-m-171763158190/
----------------------------------------
Property Name: 1650 Sq Ft Apartment With Nice Bedrooms For Sale In M

In [232]:
import pickle
# Save the model to a file
with open('preprocessor_pipeline2.pkl', 'wb') as file:
    pickle.dump(preprocessor, file)


In [233]:
import pickle

# Save similarity matrix
with open(r"D:\Work file\bd_real_estate\models\similarity_matrix2.pkl", 'wb') as f:
    pickle.dump(similarity_matrix, f)


In [18]:
# Get the counts of each unique 'area' value
area_counts = df["area"].value_counts()

# Filter the rows where 'area' appears 20 or more times
df = df[df["area"].isin(area_counts[area_counts >= 20].index)]



In [19]:
df["area"].value_counts()

area
Mirpur             1565
Bashundhara R-A     876
Uttara              605
Badda               448
Mohammadpur         341
Banasree            284
Agargaon            111
Rampura             103
Khilgaon             87
Tejgaon              57
Motijheel            54
Gulshan              48
Dakshin Khan         39
Mohakhali            32
Keraniganj           30
Aftab Nagar          28
New Market           27
Banani               24
Dhanmondi            21
Name: count, dtype: int64

In [20]:
df.to_csv('recomendation.csv')

In [152]:
rf=pd.read_csv('recomendation.csv')

In [153]:
rf.shape

(4780, 14)

In [180]:
af=pd.read_csv('Dhanmondi.csv')

In [181]:
rf = pd.concat([rf,af],ignore_index=True)

In [201]:
rf.isnull().sum()

property_name        0
address              0
short_description    0
price                0
bedrooms             0
bathrooms            0
floor_area           0
property_url         0
type                 0
teg                  0
price_per_sqft       0
sub_area             0
area                 0
dtype: int64

In [193]:
rf = rf.dropna(subset=['bedrooms', 'bathrooms','area'])


In [200]:
rf=rf.drop(columns='Unnamed: 0',axis=1)

In [184]:
rf['floor_area'] = rf['floor_area'].replace('[ sqft]', '', regex=True).astype(float)
rf['price'] = rf['price'].replace('[৳,]', '', regex=True).astype(float)

In [185]:
rf['short_description'] = rf['short_description'].str.lower()

In [186]:
rf["short_description"] = rf["short_description"].str.replace(r'[^A-Za-z0-9 ]', '', regex=True)

In [187]:
rf["short_description"] = rf["short_description"].apply(lambda x: ' '.join(str(x).split()))


In [188]:
rf.duplicated().sum()

np.int64(4)

In [189]:
def calculate_price_per_sqft(price, floor_area):
    if floor_area <= 0:
        raise ValueError("Floor area must be greater than zero.")
    return price / floor_area

rf['price_per_sqft'] = rf.apply(lambda x: calculate_price_per_sqft(x['price'], x['floor_area']), axis=1)

In [190]:
rf[['sub_area', 'area']] = rf['address'].str.split(',', expand=True)

In [195]:
rf['area'].value_counts()

area
Mirpur             1565
Bashundhara R-A     876
Uttara              605
Badda               448
Mohammadpur         341
Banasree            284
Aftab Nagar         216
Dakshin Khan        205
Dhanmondi           149
Agargaon            111
Rampura             103
Khilgaon             87
Tejgaon              57
Motijheel            54
Gulshan              48
Mohakhali            32
Keraniganj           30
New Market           27
Banani               24
Name: count, dtype: int64

In [202]:
rf.to_csv("Recommendation_data.csv")